<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/orpheus_tokenise_speech_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets snac


In [ ]:
from google.colab import userdata
TOKEN=userdata.get('HF_TOKEN')
# Change these

my_original_dataset_name = "canopylabs/zac-sample-dataset"




## CHANGE TO YOUR HUGGINGFACE TOKEN
!huggingface-cli login --token=$TOKEN --add-to-git-credential

In [5]:
#@title Installation & Setup
%%capture
import locale
locale.getpreferredencoding = lambda: "UTF-8"
import torch
from snac import SNAC
from datasets import load_dataset
from huggingface_hub import snapshot_download
from datasets import load_dataset

dsn = my_original_dataset_name

snapshot_download(
    repo_id=dsn,
    repo_type="dataset",
    revision="main",
    max_workers=64,
)


ds = load_dataset(dsn, split="train")
ds_sample_rate = ds[0]["audio"]["sampling_rate"]

model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz")
model = model.to("cuda")

In [12]:
print(ds_sample_rate)

48000


In [6]:
#@title Tokenisation Function
import torchaudio.transforms as T
def tokenise_audio(waveform):
  waveform = torch.from_numpy(waveform).unsqueeze(0)
  waveform = waveform.to(dtype=torch.float32)
  resample_transform = T.Resample(orig_freq=ds_sample_rate, new_freq=24000)
  waveform = resample_transform(waveform)

  waveform = waveform.unsqueeze(0).to("cuda")

  #generate the codes from snac
  with torch.inference_mode():
    codes = model.encode(waveform)

  all_codes = []
  for i in range(codes[0].shape[1]):
    all_codes.append(codes[0][0][i].item()+128266)
    all_codes.append(codes[1][0][2*i].item()+128266+4096)
    all_codes.append(codes[2][0][4*i].item()+128266+(2*4096))
    all_codes.append(codes[2][0][(4*i)+1].item()+128266+(3*4096))
    all_codes.append(codes[1][0][(2*i)+1].item()+128266+(4*4096))
    all_codes.append(codes[2][0][(4*i)+2].item()+128266+(5*4096))
    all_codes.append(codes[2][0][(4*i)+3].item()+128266+(6*4096))


  return all_codes




In [7]:
#@title Map Tokenize
import random
def add_codes(example):
    # Always initialize codes_list to None
    codes_list = None

    try:
        answer_audio = example.get("audio")
        # If there's a valid audio array, tokenise it
        if answer_audio and "array" in answer_audio:
            audio_array = answer_audio["array"]
            codes_list = tokenise_audio(audio_array)
    except Exception as e:
        print(f"Skipping row due to error: {e}")
        # Keep codes_list as None if we fail
    example["codes_list"] = codes_list

    return example

ds = ds.map(add_codes, remove_columns=["audio"])


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [11]:
print(ds[0])

{'text': 'Jake, have you started packing for your camping trip this weekend?', 'codes_list': [130334, 132788, 137439, 142067, 147667, 149534, 153416, 131482, 134788, 138037, 141514, 145086, 149250, 154338, 132085, 133025, 139026, 144113, 144688, 151164, 154862, 129332, 132393, 136849, 141599, 145340, 149760, 155307, 131068, 133365, 136960, 142137, 146722, 149013, 156318, 132085, 134177, 137711, 144522, 146447, 151983, 155913, 129745, 132938, 137503, 140757, 146548, 151818, 156549, 129535, 133218, 137005, 144151, 145105, 150587, 154067, 130703, 133860, 138072, 144497, 148222, 152410, 153102, 128270, 136032, 139386, 140761, 145865, 149191, 156499, 131948, 133604, 137782, 142527, 146478, 150701, 153196, 131720, 134387, 139847, 143528, 147584, 152403, 153287, 129103, 132621, 137738, 142973, 145745, 148783, 155925, 131449, 133160, 137584, 143245, 148530, 149604, 156908, 131813, 134370, 137866, 144245, 144920, 150148, 153617, 130136, 132518, 140275, 144584, 148269, 149890, 156336, 130489, 13

In [13]:
#@title Load Tokenizer
tokeniser_length = 128256
start_of_text = 128000
end_of_text = 128009

start_of_speech = tokeniser_length + 1
end_of_speech = tokeniser_length + 2

start_of_human = tokeniser_length + 3
end_of_human = tokeniser_length + 4

start_of_ai = tokeniser_length + 5
end_of_ai =  tokeniser_length + 6
pad_token = tokeniser_length + 7

audio_tokens_start = tokeniser_length + 10

tokenizer_name = "canopylabs/orpheus-3b-0.1-pretrained"

from transformers import AutoTokenizer
import os
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
num_proc = os.cpu_count() - 2

ds = ds.filter(lambda x: x["codes_list"] is not None)
ds = ds.filter(lambda x: len(x["codes_list"]) > 0)

tokenizer_config.json:   0%|          | 0.00/5.41M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/22.8M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Filter:   0%|          | 0/20 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20 [00:00<?, ? examples/s]

In [15]:
print(ds[1])

{'text': 'My Olympic dreams are over. I missed the qualifying time by less than a second.', 'codes_list': [131503, 134893, 137809, 142557, 147535, 151664, 155853, 132028, 133163, 136721, 141208, 147668, 151212, 154029, 132179, 136048, 137660, 141077, 147203, 151459, 153344, 132273, 134858, 137232, 144519, 147203, 152711, 156306, 129137, 133456, 137864, 142274, 146920, 150698, 156738, 132179, 132892, 138911, 141140, 144985, 151056, 154091, 132179, 134571, 139143, 144369, 146557, 150601, 156367, 132273, 133786, 139416, 142427, 146553, 152178, 156182, 128913, 136438, 137839, 144335, 146225, 151617, 156045, 131735, 132638, 139998, 143932, 144832, 152134, 156770, 129326, 133008, 138269, 144077, 144939, 152356, 155159, 130426, 136282, 137318, 143064, 146560, 151433, 153394, 128466, 134272, 138068, 141799, 145500, 151342, 154620, 131873, 135158, 137548, 141586, 145342, 150392, 155269, 129894, 134745, 139496, 143261, 147037, 152482, 154639, 130102, 133185, 138421, 143232, 146087, 149947, 15663

In [ ]:
#@title Create Input Ids
def remove_duplicate_frames(example):
    vals = example["codes_list"]
    if len(vals) % 7 != 0:
        raise ValueError("Input list length must be divisible by 7")

    result = vals[:7]

    removed_frames = 0

    for i in range(7, len(vals), 7):
        current_first = vals[i]
        previous_first = result[-7]

        if current_first != previous_first:
            result.extend(vals[i:i+7])
        else:
            removed_frames += 1

    example["codes_list"] = result

    return example

ds = ds.map(remove_duplicate_frames, num_proc=num_proc)

tok_info = '''*** HERE you can modify the text prompt
i.e. if you wanted a multispeaker model like canopylabs/orpheus-3b-0.1-ft, you can pass:
f"{example["source"]}:  {example["text"]}", as is passed.
'''
print(tok_info)

def create_input_ids(example):
    text_ids = tokenizer.encode(example["text"],  add_special_tokens=True)
    text_ids.append(end_of_text)
    example["text_tokens"] = text_ids
    input_ids = (
        [start_of_human]
        + example["text_tokens"]
        + [end_of_human]
        + [start_of_ai]
        + [start_of_speech]
        + example["codes_list"]
        + [end_of_speech]
        + [end_of_ai]
    )
    example["input_ids"] = input_ids
    example["labels"] = input_ids
    example["attention_mask"] = [1] * len(input_ids)

    return example

ds = ds.map(create_input_ids, num_proc=num_proc, remove_columns=["text", "codes_list"])


In [20]:
print(ds)
print(ds[0]['text_tokens'])
print(ds[0]['input_ids'])
print(ds[0]['labels'])
print(ds[0]['attention_mask'])


Dataset({
    features: ['text_tokens', 'input_ids', 'labels', 'attention_mask'],
    num_rows: 20
})
[128000, 95336, 11, 617, 499, 3940, 36813, 369, 701, 33873, 8577, 420, 9178, 30, 128009]
[128259, 128000, 95336, 11, 617, 499, 3940, 36813, 369, 701, 33873, 8577, 420, 9178, 30, 128009, 128260, 128261, 128257, 130334, 132788, 137439, 142067, 147667, 149534, 153416, 131482, 134788, 138037, 141514, 145086, 149250, 154338, 132085, 133025, 139026, 144113, 144688, 151164, 154862, 129332, 132393, 136849, 141599, 145340, 149760, 155307, 131068, 133365, 136960, 142137, 146722, 149013, 156318, 132085, 134177, 137711, 144522, 146447, 151983, 155913, 129745, 132938, 137503, 140757, 146548, 151818, 156549, 129535, 133218, 137005, 144151, 145105, 150587, 154067, 130703, 133860, 138072, 144497, 148222, 152410, 153102, 128270, 136032, 139386, 140761, 145865, 149191, 156499, 131948, 133604, 137782, 142527, 146478, 150701, 153196, 131720, 134387, 139847, 143528, 147584, 152403, 153287, 129103, 132621, 

In [21]:
#@title Remove unnecessary columns
columns_to_keep = ["input_ids", "labels", "attention_mask"]
columns_to_remove = [col for col in ds.column_names if col not in columns_to_keep]

ds = ds.remove_columns(columns_to_remove)

In [23]:
print(ds)
print(ds[0])

Dataset({
    features: ['input_ids', 'labels', 'attention_mask'],
    num_rows: 20
})
{'input_ids': [128259, 128000, 95336, 11, 617, 499, 3940, 36813, 369, 701, 33873, 8577, 420, 9178, 30, 128009, 128260, 128261, 128257, 130334, 132788, 137439, 142067, 147667, 149534, 153416, 131482, 134788, 138037, 141514, 145086, 149250, 154338, 132085, 133025, 139026, 144113, 144688, 151164, 154862, 129332, 132393, 136849, 141599, 145340, 149760, 155307, 131068, 133365, 136960, 142137, 146722, 149013, 156318, 132085, 134177, 137711, 144522, 146447, 151983, 155913, 129745, 132938, 137503, 140757, 146548, 151818, 156549, 129535, 133218, 137005, 144151, 145105, 150587, 154067, 130703, 133860, 138072, 144497, 148222, 152410, 153102, 128270, 136032, 139386, 140761, 145865, 149191, 156499, 131948, 133604, 137782, 142527, 146478, 150701, 153196, 131720, 134387, 139847, 143528, 147584, 152403, 153287, 129103, 132621, 137738, 142973, 145745, 148783, 155925, 131449, 133160, 137584, 143245, 148530, 149604, 15

In [ ]:
name_to_push_dataset_to = "weege007/zac_sample-dataset-tokenised"

ds.push_to_hub(name_to_push_dataset_to)